In [ ]:
%%sh
wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
unzip ml-latest-small.zip

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


--2020-09-29 19:43:42--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

     0K .......... .......... .......... .......... ..........  5%  544K 2s
    50K .......... .......... .......... .......... .......... 10% 1.06M 1s
   100K .......... .......... .......... .......... .......... 15% 86.0M 1s
   150K .......... .......... .......... .......... .......... 20% 1.07M 1s
   200K .......... .......... .......... .......... .......... 26%  236M 1s
   250K .......... .......... .......... .......... .......... 31% 65.2M 0s
   300K .......... .......... .......... .......... .......... 36% 1.08M 0s
   350K .......... .......... .......... .......... .......... 41%  104M 0s
   400K .......... ..

In [ ]:
import os 
from google.colab import drive
drive.mount("/content/drive")
os.chdir('/content/drive/Shared drives/IDS 576/Assignment 2')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import itertools
from operator import eq
import scipy.sparse as sp
import pickle

movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv').sort_values(by='movieId')
ratings['movieId'] = ratings['movieId']
ratings['movieId'] = ratings['movieId'].replace({v: k for k, v in dict(movies['movieId']).items()})
ratings_slice = ratings[['userId','movieId']]

In [ ]:
len(ratings['movieId'].unique())

9724

In [ ]:
def data_workflow(df,filename=None):
  x_dict = dict()
  for movieid in np.sort(df['movieId'].unique()):
    x_dict[movieid] = df.groupby('movieId').get_group(movieid)['userId'].values
  
  print('Grouping Done..!!')
  movie_pairs = list(itertools.combinations(x_dict.keys(), 2))

  count=0
  X_ij = dict()
  for pair in movie_pairs:
    count+=1
    common = len(set(x_dict[pair[0]])-set(x_dict[pair[1]]))
    if common!=0:X_ij[pair] = common
    if count%100000000==0:print(str(count)+' Pairs Done')
  print("Users Counted..!!")

  rows,cols,data = [i[0] for i in X_ij.keys()],[i[1] for i in X_ij.keys()],[i for i in X_ij.values()]
  Xij = sp.csr_matrix((data, (rows, cols)))
  return Xij

In [ ]:
import time

start = time.time()
sparse_Xij = data_workflow(ratings_slice)
end = time.time()
print(end-start)
sparse_Xij

Grouping Done..!!
Users Counted..!!
403.55619406700134


<9741x9742 sparse matrix of type '<class 'numpy.longlong'>'
	with 46220849 stored elements in Compressed Sparse Row format>

In [ ]:
sp.save_npz('sparse_Xij.npz', sparse_Xij)

In [ ]:
sparse_Xij = sp.load_npz('sparse_Xij.npz')

In [ ]:
np_Xij = sparse_Xij.toarray()
np_Xij = np.vstack([np_Xij,np.zeros((1,9742))])

In [ ]:
import torch
from torch import nn, optim
from torch.utils import data
import torch.nn.functional as f
xij = np_Xij
torch_xij = f.normalize(torch.tensor(xij,requires_grad=False))

###Cost function

In [ ]:


#brute implementation not efficient
#def cost_func(vi,vj,x,i,j):
#  vivj = torch.dot(vi,vj)
#  squared = torch.square(vivj-x[i,j])
#  return torch.sum(squared)

#Cost Function:
  #parameters:initial embeddings value in matrix form of size M x (embedding size)
  #xij: square matrix of size M x M

def cost_func(parameters,xij):
  vivj = torch.matmul(parameters,torch.transpose(parameters,0,1)) # parameter matrix multiplication which gives M x M matrix 
  vivij = f.normalize(torch.triu(vivj,diagonal=1))
  #idx = np.diag_indices(vivj.shape[0])
  #vivj[idx[0],idx[1]] = torch.zeros(vivj.shape[0],dtype=torch.float32) #populating zero at diagonals to make values of i=j zero
  squared = torch.pow(vivj-xij,2) #subtracting both matrix and taking square
  return torch.sum(squared) # taking sum of all the elements of M x M matrix


### Functions for optimization(SGD, Adam, Adadelta), contains learning rates too

In [ ]:
embedding_size = 100 
iter = dict()
learn_rate = np.array([1e-6,1e-7])                                              #learning rates
def optim_sgd(learn_rate):
   
   iter = []
   los = []
   params = [torch.rand(embedding_size,requires_grad=True) for i in range(9742)]
   for r in learn_rate: 
    opt = optim.SGD(params, lr=r)
    for e in range(2000):
       opt.zero_grad()
       loss = cost_func(torch.stack(params),torch_xij)
       loss.backward()
       if e%1000==0:
         print('Step # {}, loss: {:.4f}'.format(e, loss))
       elif e%100 ==0:  
         iter.append(e)
         los.append(loss.detach().numpy())
       opt.step()
   return iter,los

def optim_adam(learn_rate):
   
   iter = []
   los = []
   params = [torch.rand(embedding_size,requires_grad=True) for i in range(9742)]
   for r in learn_rate: 
    opt = optim.Adam(params, lr=r)
    for e in range(2000):
       opt.zero_grad()
       loss = cost_func(torch.stack(params),torch_xij)
       loss.backward()
       if e%1000==0:
         print('Step # {}, loss: {:.4f}'.format(e, loss))
       elif e%100 ==0: 
         iter.append(e)
         los.append(loss.detach().numpy())
       opt.step()
   return iter,los

def optim_adadelta(learn_rate):
                   
   iter = []
   los = []
   params = [torch.rand(embedding_size,requires_grad=True) for i in range(9742)]
   for r in learn_rate: 
    opt = optim.Adadelta(params, lr=r)
    for e in range(2000):
       opt.zero_grad()
       loss = cost_func(torch.stack(params),torch_xij)
       loss.backward()
       if e%1000==0:
         print('Step # {}, loss: {:.4f}'.format(e, loss))
       elif e%100 ==0: 
         iter.append(e)
         los.append(loss.detach().numpy())
       opt.step()
   return iter,los

###Function to plot losses vs iterations

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

def plot_losses(iteration, loss, title, learn_rate):
 iteration = np.split(np.array(iterations),len(learn_rate))  
 loss = np.split(np.array(losses),len(learn_rate))
 
 for val in range(len(learn_rate)): 
   fig = plt.figure(val)
   plt.plot(iteration[val],loss[val])   
   #plt.figure(val)
   plt.title(learn_rate[val])
   fig.suptitle(title)
   plt.xlabel('iterations')
   plt.ylabel('losses')
   plt.show()

### Loss optimization using SGD

In [ ]:
##Loss as a function of iteration for various choices
                                                     
iterations,losses = optim_sgd(learn_rate)                                        #SGD optimization
plot_losses(iterations, losses,'SGD', learn_rate)


Step # 0, loss: 59842218750.0709


### Loss optimization using Adam

In [ ]:
iterations,losses = optim_adam(learn_rate)                                       #Adam optimization
plot_losses(iterations, losses, 'Adam', learn_rate)

 

### Loss optimization using Adadelta

In [ ]:
iterations,losses = optim_adadelta(learn_rate)                                             #Adadelta optimization
plot_losses(iterations, losses, 'Adadelta', learn_rate)